In [1]:
import tensorflow as tf

import matplotlib as mpl
import matplotlib.pyplot as plt

# lya_fields.py

In [2]:
import numpy as np
import tensorflow as tf

# local modules
import grid
import lya_fields
import snapshot
from spectrum import gmlt_spec_od_grid

# define dataset paths
ds_path_rhob = "/native_fields/baryon_density"
ds_path_temp = "/native_fields/temperature"
ds_path_vx   = "/native_fields/velocity_x"
ds_path_vy   = "/native_fields/velocity_y"
ds_path_vz   = "/native_fields/velocity_z"

ds_path_nhi      = "/derived_fields/HI_number_density"
ds_path_tau_real = "/derived_fields/tau_real"
ds_path_tau_red  = "/derived_fields/tau_red"

filename = "../../../../../cscratch1/sd/jupiter/sim2_z3_FGPA_cgs.h5"
snap = snapshot.Snapshot(filename)

## Compute $n_\text{H I}$ (1024^2 grid)

In [3]:
# subsection shape
shape = [1, 1024, 1024]

rhob = snap.read_subfield(ds_path_rhob, shape)
temp = snap.read_subfield(ds_path_temp, shape)

In [4]:
# get the nhi grid
with tf.GradientTape(persistent=True) as tape:
    
#     tape.watch(rhob.field)
#     tape.watch(temp.field)

    nhi = lya_fields.set_nhi(snap, rhob, temp)

EOS duration: 7.031138181686401


In [5]:
#print("nhi's shape:", tf.shape(nhi.field))
dn_dr = tape.gradient(nhi.field, rhob.field)
dn_dt = tape.gradient(nhi.field, temp.field)

interp.dfitpack.bispeu works!
fake_rho: tf.Tensor([-20 -19], shape=(2,), dtype=int32)
fake_rho type: <class 'tensorflow.python.framework.ops.EagerTensor'>
fake_rho shape: tf.Tensor([2], shape=(1,), dtype=int32)
compute_dn_dlogx works on 1D tensors!
interp.dfitpack.bispeu works!
fake_rho: tf.Tensor([-20 -19], shape=(2,), dtype=int32)
fake_rho type: <class 'tensorflow.python.framework.ops.EagerTensor'>
fake_rho shape: tf.Tensor([2], shape=(1,), dtype=int32)
compute_dn_dlogx works on 1D tensors!


### test jacobian

In [6]:
type(tf.constant(1))

tensorflow.python.framework.ops.EagerTensor

In [8]:
# this throws an error
dn_dr_j = tape.batch_jacobian(nhi.field, rhob.field, experimental_use_pfor=True)

# this prints a None gradient
# flat_nhi = tf.reshape(nhi.field, [1024**2])
# flat_rhob = tf.reshape(rhob.field, [1024**2])
# dn_dr_j = tape.jacobian(flat_nhi, rhob.field)

print(dn_dr_j)

Error: interp.dfitpack.bispeu doesn't work
<class 'TypeError'>
fake_rho: Tensor("loop_body/Const:0", shape=(2,), dtype=int32)
fake_rho type: <class 'tensorflow.python.framework.ops.Tensor'>
fake_rho shape: Tensor("loop_body/Shape:0", shape=(1,), dtype=int32)
Error: compute_dn_dlogx doesn't work on 1D tensors
<class 'TypeError'>


TypeError: in user code:

    /global/homes/j/jupiter/.conda/envs/lya-tf/lib/python3.7/site-packages/tensorflow/python/ops/parallel_for/control_flow_ops.py:188 f  *
        iters,
    /global/homes/j/jupiter/.conda/envs/lya-tf/lib/python3.7/site-packages/tensorflow/python/ops/parallel_for/control_flow_ops.py:248 _pfor_impl  **
        loop_fn_outputs = loop_fn(loop_var)
    /global/homes/j/jupiter/.conda/envs/lya-tf/lib/python3.7/site-packages/tensorflow/python/eager/backprop.py:1324 loop_fn
        unconnected_gradients=unconnected_gradients)
    /global/homes/j/jupiter/.conda/envs/lya-tf/lib/python3.7/site-packages/tensorflow/python/eager/backprop.py:1086 gradient
        unconnected_gradients=unconnected_gradients)
    /global/homes/j/jupiter/.conda/envs/lya-tf/lib/python3.7/site-packages/tensorflow/python/eager/imperative_grad.py:77 imperative_grad
        compat.as_str(unconnected_gradients.value))
    /global/homes/j/jupiter/.conda/envs/lya-tf/lib/python3.7/site-packages/tensorfl

In [ ]:
def print_minmax(t, name):
    '''
    Given a tensor t and its name, print t's minimum and maximum values.
    
    '''
    
    print(name + ' min:', np.amin(t))
    print(name + ' max:', np.amax(t))
    print('')

In [ ]:
print_minmax(dn_dr, 'dn/dr')
print_minmax(dn_dt, 'dn/dt')

## Look at the $\frac{dn}{d\rho}$ and $\frac{dn}{dT}$ fields

In [ ]:
fig = plt.figure(figsize=(8,8))
plt.imshow(np.log10(dn_dr[0,:,:]), cmap='Blues')
plt.colorbar()
plt.title(r'log10(dn/d$\rho$)')
plt.show()

fig = plt.figure(figsize=(8,8))
plt.imshow(np.log10(-dn_dt[0,:,:]), cmap='inferno')
plt.colorbar()
plt.title('log10(-dn/dT)')
plt.show()

## Compute $\tau_\text{real}$ and $\tau_\text{red}$ (one skewer)

In [ ]:
# subsection shape
shape = [1, 1, 1024]

rhob = snap.read_subfield(ds_path_rhob, shape)
temp = snap.read_subfield(ds_path_temp, shape)

# get the tau grids
with tf.GradientTape(persistent=True) as tape:
#     tape.watch(rhob.field)
#     tape.watch(temp.field)

    # number density
    nhi = lya_fields.set_nhi(snap, rhob, temp)
    
    # real-space tau
    vpara = grid.Grid(tf.zeros(rhob.shape, dtype='float64'), rhob.shape, rhob.size)
    tau_real = gmlt_spec_od_grid(snap.universe, snap.z, nhi.size,
            nhi.field, temp.field, vpara.field, nhi.field.shape[2])

    # redshift-space tau
    vpara = snap.read_subfield(ds_path_vz, shape)
    tau_red = gmlt_spec_od_grid(snap.universe, snap.z, nhi.size,
            nhi.field, temp.field, vpara.field, nhi.field.shape[2])

## tape.gradient vs. tape.jacobian

The $i$th element in `tape.gradient(tau_real.field, rhob.field)` is the gradient of the sum of all $\tau$s with respect to $\rho_i$, which isn't what we want.

To get $\frac{\partial \tau_i}{\partial x_j}$ for every pair $i,j$, we need the Jacobian.

## Look at the 4 Jacobians

In [ ]:
dtreal_dr_j = tape.jacobian(tau_real.field, rhob.field)
# dtreal_dt_j = tape.jacobian(tau_real.field, temp.field)

# dtred_dr_j = tape.jacobian(tau_red.field, rhob.field)
# dtred_dt_j = tape.jacobian(tau_red.field, temp.field)

In [ ]:
# this returns the same error as above
dtreal_dn_j = tape.jacobian(tau_real.field, nhi.field)
print(dtreal_dn)

In [ ]:
dtreal_dr = tape.gradient(tau_real.field, rhob.field)
print(dtreal_dr)

In [ ]:
# dtreal_dr = tape.gradient(tau_real.field, rhob.field)
# dtreal_dt = tape.gradient(tau_real.field, temp.field)

# dtred_dr = tape.gradient(tau_red.field, rhob.field)
# dtred_dt = tape.gradient(tau_red.field, temp.field)

# print_minmax(dtreal_dr, 'dtreal/dr')
# print_minmax(dtreal_dt, 'dtreal/dt')

# print_minmax(dtred_dr, 'dtred/dr')
# print_minmax(dtred_dt, 'dtred/dt')

### Look at the 4 $\frac{d\Sigma \tau}{dx_i}$ fields (tape.gradient)

In [ ]:
figsize=(16,4)

section = slice(0,80)

vmin_r = -0.5
vmax_r = 0.5
vmin_t = 2e-10
vmax_t = 7e-10

fig = plt.figure(figsize=figsize)
plt.imshow(np.log10(dtreal_dr[0, :, section]), cmap='Blues', vmin=vmin_r, vmax=vmax_r)
plt.colorbar()
plt.title(r'log10($d\tau_{real} / d\rho$) (left)')
plt.show()

fig = plt.figure(figsize=figsize)
plt.imshow(np.log10(dtred_dr[0, :, section]), cmap='Blues', vmin=vmin_r, vmax=vmax_r)
plt.colorbar()
plt.title(r'log10($d\tau_{red} / d\rho$) (left)')
plt.show()

fig = plt.figure(figsize=figsize)
plt.imshow(-dtreal_dt[0, :, section], cmap='inferno', vmin=vmin_t, vmax=vmax_t)
plt.colorbar()
plt.title(r'-$d\tau_{real} / dT$ (left)')
plt.show()

fig = plt.figure(figsize=figsize)
plt.imshow(-dtred_dt[0, :, section], cmap='inferno', vmin=vmin_t, vmax=vmax_t)
plt.colorbar()
plt.title(r'-$d\tau_{red} / dT$ (left)')
plt.show()

# Jacobian examples

In [ ]:
# x = tf.linspace(-10.0, 10.0, 200+1)
# delta = tf.Variable(0.0)

# with tf.GradientTape() as tape:
#     y = tf.nn.sigmoid(x+delta)

# dy_dx = tape.jacobian(y, delta)

# plt.plot(x.numpy(), y, label='y')
# plt.plot(x.numpy(), dy_dx, label='dy/dx')
# plt.legend()
# _ = plt.xlabel('x')

[Define jacobian with tf.custom_gradient](https://stackoverflow.com/questions/66536946/define-jacobian-with-tensorflow-custom-gradient)

In [5]:
@tf.custom_gradient
def f(x):
    y = tf.stack([x[1], 2.0*x[0]])

    def grad(upstream):
        return tf.stack([2.0*upstream[1], upstream[0]])

    return y, grad

x = tf.Variable([1.0, 2.0])
with tf.GradientTape() as tape:
    y = f(x)
tape.jacobian(y, x)
#tape.gradient(y,x)

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[0., 1.],
       [2., 0.]], dtype=float32)>